<a href="https://colab.research.google.com/github/arifsl65/AI/blob/main/Youtube_transcript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# youtube transcript pdf

!pip install fpdf youtube-transcript-api
!pip install openai==0.28


from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi
from fpdf import FPDF

def extract_video_id(url):
    parsed_url = urlparse(url)
    if parsed_url.netloc == 'www.youtube.com':
        query_params = parse_qs(parsed_url.query)
        return query_params.get('v', [None])[0]
    elif parsed_url.netloc == 'youtu.be':
        return parsed_url.path.lstrip('/')
    return None

def get_youtube_transcript(video_id):
    try:
        # Fetch the transcript using the video ID
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        # Combine the transcript into a single string
        transcript_text = "\n".join([entry['text'] for entry in transcript])
        return transcript_text
    except Exception as e:
        return f"Error: {str(e)}"

def save_transcript_as_pdf(transcript_text, file_name):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    # Add the transcript text to the PDF
    for line in transcript_text.split("\n"):
        pdf.multi_cell(0, 10, line)

    # Save the PDF with the specified filename
    pdf.output(file_name)
    print(f"Transcript saved as {file_name}")

def main():
    # Take URL as user input
    url = input("Enter the YouTube URL: ")

    # Extract video ID from the URL
    video_id = extract_video_id(url)

    if video_id:
        # Fetch the transcript
        transcript = get_youtube_transcript(video_id)

        # Check if there's an error in fetching the transcript
        if transcript.startswith("Error"):
            print(transcript)
        else:
            # Save the transcript as a PDF
            save_transcript_as_pdf(transcript, "transcript.pdf")
    else:
        print("Invalid YouTube URL")

if __name__ == "__main__":
    main()


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=db1b746a48270ce6dfd7cd0abfac345666afeb5fa1e99d514deca5080177885c
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 845.3 kB/s eta 0:00:00
Enter the YouTube URL: https://youtu.be/guXTAOcrZaY?si=dlti44JZjyqtCLK-
Transcript saved as transcript.pdf


In [24]:

# summarise transcript with openai
import openai
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

# Set your OpenAI API key
openai.api_key = api_key

def extract_video_id(url):
    parsed_url = urlparse(url)

    if parsed_url.netloc == 'www.youtube.com':
        query_params = parse_qs(parsed_url.query)
        return query_params.get('v', [None])[0]
    elif parsed_url.netloc == 'youtu.be':
        return parsed_url.path.lstrip('/')

    return None

def get_youtube_transcript(video_id):
    try:
        # Fetch the transcript using the video ID
        transcript = YouTubeTranscriptApi.get_transcript(video_id)

        # Combine the transcript into a single string
        transcript_text = "\n".join([entry['text'] for entry in transcript])

        return transcript_text

    except Exception as e:
        return f"Error: {str(e)}"

def summarize_transcript(transcript):
    prompt = (
        "Summarize the following transcript into key points:\n\n"
        f"{transcript}\n\n"
        "Key Points:"
    )

    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",  # Choose a supported engine
        prompt=prompt,
        temperature=0.5,  # Moderate randomness for more focused output
        max_tokens=300,  # Adjust based on the length of expected summary
        top_p=1.0,  # Controls the diversity of the output
        frequency_penalty=0.0,  # Controls how much the model penalizes new information
        presence_penalty=0.0  # Controls how much the model penalizes the presence of new information
    )

    return response.choices[0].text.strip()

def main():
    # Take URL as user input
    url = input("Enter the YouTube URL: ")

    # Extract video ID from the URL
    video_id = extract_video_id(url)

    if video_id:
        # Fetch the transcript
        transcript = get_youtube_transcript(video_id)

        if transcript.startswith("Error:"):
            print(transcript)
        else:
            # Summarize the transcript with key points
            summary = summarize_transcript(transcript)
            print("\nTranscript Summary with Key Points:")
            print(summary)
    else:
        print("Invalid YouTube URL")

if __name__ == "__main__":
    main()


Enter the YouTube URL: https://youtu.be/gbp0nU0WR1w?si=UO7TxL8b1kQJW40u


InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 23868 tokens (23568 in your prompt; 300 for the completion). Please reduce your prompt; or completion length.